<a href="https://colab.research.google.com/github/jvishnuvardhan/Stackoverflow_Questions/blob/master/Save_load_model_with_customeObjects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tf-nightly

     |████████████████████████████████| 517.1MB 30kB/s 
     |████████████████████████████████| 460kB 40.2MB/s 
     |████████████████████████████████| 2.9MB 39.4MB/s 


In [4]:
import tensorflow as tf
from tensorflow import keras
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Custom Loss1 (for example) 
@tf.function() 
def customLoss1(yTrue,yPred):
  return tf.reduce_mean(yTrue-yPred) 

# Custom Loss2 (for example) 
@tf.function() 
def customLoss2(yTrue, yPred):
  return tf.reduce_mean(tf.square(tf.subtract(yTrue,yPred))) 
  
def create_model():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),  
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', customLoss1, customLoss2])
  return model

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
# Create a basic model instance
model=create_model()

# Fit and evaluate model 
model.fit(x_train, y_train, epochs=5)
loss, acc,loss1, loss2 = model.evaluate(x_test, y_test,verbose=1)
print("Original model, accuracy: {:5.2f}%".format(100*acc))

Epoch 1/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2179 - accuracy: 0.9356 - customLoss1: 4.3539 - customLoss2: 27.3820
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0945 - accuracy: 0.9712 - customLoss1: 4.3539 - customLoss2: 27.3892
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0696 - accuracy: 0.9783 - customLoss1: 4.3539 - customLoss2: 27.3907
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0537 - accuracy: 0.9826 - customLoss1: 4.3539 - customLoss2: 27.3915
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0690 - accuracy: 0.9783 - customLoss1: 4.3429 - customLoss2: 27.3308
Original model, accuracy: 97.83%


In [0]:
# save the model in *.h5 format
model.save('./MyModel.h5',save_format='h5')

In [0]:
# load the saved model (Don't compile while loading as custom objects are not serialized)
loaded_model = tf.keras.models.load_model('./MyModel.h5', custom_objects={customLoss1:customLoss1, customLoss2:customLoss2},compile=False)

In [0]:
# compile the loaded model with the custome objects
loaded_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', customLoss1, customLoss2])